In [7]:
import pandas as pd
import numpy as np
import torch

In [49]:
df = pd.read_csv('./torch_prac/data/train_data.csv')
df.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [87]:
sam = df.iloc[1,1:3]

In [89]:
type(sam[0])

str

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [15]:
from torch.utils.data import Dataset, ConcatDataset, Sampler, RandomSampler, BatchSampler

In [8]:
torch.rand(10)

tensor([0.3984, 0.6070, 0.5832, 0.4742, 0.8500, 0.0263, 0.7227, 0.2417, 0.5673,
        0.3246])

In [9]:
class MapDataset(Dataset):
    def __len__(self):
        return 10
    
    def __getitem__(self, idx):
        return {"input":torch.tensor([idx, 2*idx, 3*idx], 
                                     dtype=torch.float32), 
                "label": torch.tensor(idx, 
                                      dtype=torch.float32)}

map_dataset = MapDataset()

In [12]:
dataloader = torch.utils.data.DataLoader(map_dataset)
for data in dataloader:
    print(data['input'])

tensor([[0., 0., 0.]])
tensor([[1., 2., 3.]])
tensor([[2., 4., 6.]])
tensor([[3., 6., 9.]])
tensor([[ 4.,  8., 12.]])
tensor([[ 5., 10., 15.]])
tensor([[ 6., 12., 18.]])
tensor([[ 7., 14., 21.]])
tensor([[ 8., 16., 24.]])
tensor([[ 9., 18., 27.]])


In [13]:
dataloader = torch.utils.data.DataLoader(map_dataset,
                                         batch_size=4)
for data in dataloader:
    print(data['input'].shape, data['label'])

torch.Size([4, 3]) tensor([0., 1., 2., 3.])
torch.Size([4, 3]) tensor([4., 5., 6., 7.])
torch.Size([2, 3]) tensor([8., 9.])


In [16]:
point_sampler = RandomSampler(map_dataset)
dataloader = torch.utils.data.DataLoader(map_dataset,
                                         batch_size=4,
                                         sampler=point_sampler)
for data in dataloader:
    print(data['input'].shape, data['label'])

torch.Size([4, 3]) tensor([0., 6., 8., 1.])
torch.Size([4, 3]) tensor([7., 5., 4., 2.])
torch.Size([2, 3]) tensor([3., 9.])


In [17]:
point_sampler = RandomSampler(map_dataset)
batch_sampler = BatchSampler(point_sampler, 3, False)
dataloader = torch.utils.data.DataLoader(map_dataset,
                                         batch_sampler=batch_sampler)
for data in dataloader:
    print(data['input'].shape, data['label'])

torch.Size([3, 3]) tensor([0., 6., 4.])
torch.Size([3, 3]) tensor([9., 7., 1.])
torch.Size([3, 3]) tensor([3., 8., 5.])
torch.Size([1, 3]) tensor([2.])


In [18]:
class VarMapDataset(Dataset):
    def __len__(self):
        return 10
    
    def __getitem__(self, idx):
        return {"input":torch.tensor([idx] * (idx+1), 
                                     dtype=torch.float32), 
                "label": torch.tensor(idx, 
                                      dtype=torch.float32)}

var_map_dataset = VarMapDataset()

In [23]:
def make_batch(samples):
    inputs = [sample['input'] for sample in samples]
    labels = [sample['label'] for sample in samples]
    padded_inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True)
    return {'input': padded_inputs.contiguous(),
            'label': torch.stack(labels).contiguous()}
    
dataloader = torch.utils.data.DataLoader(var_map_dataset,
                                         batch_size=3,
                                         collate_fn=make_batch)

In [25]:
for data in dataloader:
    print(data['label'])
    print(data['input'])

tensor([0., 1., 2.])
tensor([[0., 0., 0.],
        [1., 1., 0.],
        [2., 2., 2.]])
tensor([3., 4., 5.])
tensor([[3., 3., 3., 3., 0., 0.],
        [4., 4., 4., 4., 4., 0.],
        [5., 5., 5., 5., 5., 5.]])
tensor([6., 7., 8.])
tensor([[6., 6., 6., 6., 6., 6., 6., 0., 0.],
        [7., 7., 7., 7., 7., 7., 7., 7., 0.],
        [8., 8., 8., 8., 8., 8., 8., 8., 8.]])
tensor([9.])
tensor([[9., 9., 9., 9., 9., 9., 9., 9., 9., 9.]])


In [27]:
df_test = pd.DataFrame(np.random.randint(1,10,size= (10,3)), columns = ['user','item','score'])

In [41]:
sum(map(len,df['title']))/len(df)

27.330551539843167

In [46]:
from transformers import AutoTokenizer, AutoModel
import torch

In [47]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sbert-multitask')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')


Downloading: 100%|██████████| 538/538 [00:00<00:00, 136kB/s]
Downloading: 100%|██████████| 243k/243k [00:00<00:00, 305kB/s]  
Downloading: 100%|██████████| 483k/483k [00:01<00:00, 470kB/s]  
Downloading: 100%|██████████| 620/620 [00:00<00:00, 303kB/s]
Downloading: 100%|██████████| 422M/422M [00:44<00:00, 9.88MB/s]  


In [48]:
encoded_input

{'input_ids': tensor([[    2,  9796,  4641, 11376, 11171, 12235, 17856,  5083, 17219, 30062,
          9963,     3],
        [    2,    41, 13329, 17219, 30062,  9963, 11376, 10190,  7521, 14210,
          2039,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [83]:
from typing import Optional
import transformers
from transformers import RobertaForSequenceClassification
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification


class ROBERTA_Classifier:
    def __init__(self, config):
        self.classifier = RobertaForSequenceClassification.from_pretrained(
            "klue/roberta-large", num_labels=7
        )


In [80]:
model = ROBERTA_Classifier()

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class

In [82]:
model.classifier

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [67]:
model_test = ROBERTA_Classifier()
model_arch = model_test.init_backbone()

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class

In [72]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [54]:
model_test.init_backbone()

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class

In [75]:
model_test = RobertaForSequenceClassification.from_pretrained(
            "klue/roberta-large", num_labels=7
        )

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class

In [94]:
sentences = '안녕.'
encoded_input = tokenizer(sentences, max_length = 15, padding=True, truncation=True, return_tensors='pt')

In [95]:
encoded_input

{'input_ids': tensor([[   0, 5891,   18,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [91]:
with torch.no_grad():
    model_output = model_test(**encoded_input)

In [92]:
model_output

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0095,  0.0255, -0.0174,  0.2582,  0.0723, -0.2010,  0.1063]]), hidden_states=None, attentions=None)

In [56]:
print(model_test)

In [57]:
classifier = RobertaForSequenceClassification.from_pretrained(
            "klue/roberta-large", num_labels=7
        )

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class